<a href="https://colab.research.google.com/github/OmarovaJuma/AI.MO/blob/main/Lr9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Выполнила работу Омарова Джума группа ПИН-б-о-22-1**

# **Лабораторная работа 9. Использование разработки пайплайна для многомерной регрессии**

# **Первичный анализ**

Описание набора данных:

Набор данных содержит информацию о грибах и предназначен для бинарной классификации (съедобные или ядовитые).

Описание каждого признака и его тип:

Диаметр шляпки (cap diameter)

Форма шляпки (cap shape)

Присоединение жабер (gill attachment)

Цвет жабер (gill color)

Высота ножки (stem height)

Ширина ножки (stem width)

Цвет ножки (stem color)

Сезон (season)

Целевой класс (class) - является ли гриб съедобным (0) или ядовитым (1)

Форма набора данных:

Количество элементов: 17863

Количество признаков: 9

Количество пропущенных значений: присутствуют в последней строке для сезона и целевого класса (NaN).

Средние значения признаков:

In [ ]:
# Рассчитать статистические показатели
stats = dataset.describe()

# Вывести статистические показатели
print(stats)

       cap-diameter     cap-shape  gill-attachment    gill-color  \
count  17863.000000  17863.000000     17863.000000  17863.000000   
mean     519.719532      3.796395         2.239433      8.566758   
std      354.812248      2.239702         2.345708      2.628365   
min        0.000000      0.000000         0.000000      0.000000   
25%      242.000000      2.000000         0.000000      7.000000   
50%      495.000000      5.000000         1.000000     10.000000   
75%      733.000000      6.000000         5.000000     10.000000   
max     1866.000000      6.000000         6.000000     11.000000   

        stem-height    stem-width    stem-color        season         class  
count  17863.000000  17863.000000  17863.000000  17862.000000  17862.000000  
mean       0.804681    892.289425      8.390360      0.985202      0.451853  
std        0.705671    711.697333      3.186677      0.310950      0.497690  
min        0.000426      1.000000      0.000000      0.027372      0.000000

Импорт необходимых библиотек:

numpy для работы с массивами и матрицами;

matplotlib.pyplot для визуализации данных;

pandas для работы с данными в формате DataFrame;

LinearRegression из sklearn.linear_model для обучения линейной регрессии;

StandardScaler из sklearn.preprocessing для масштабирования признаков;

train_test_split из sklearn.model_selection для разделения данных на обучающий и тестовый наборы;

LabelEncoder и OneHotEncoder из sklearn.preprocessing для преобразования категориальных признаков в числовые;

ColumnTransformer из sklearn.compose для применения преобразований к определенным столбцам данных;

statsmodels.api для выполнения статистического анализа.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import statsmodels.api as sm

Выполняет загрузку данных из CSV-файла в формате DataFrame с помощью библиотеки Pandas. После загрузки данных они отображаются на экране с помощью метода dataset, что позволяет быстро взглянуть на структуру и содержание данных.

In [ ]:
dataset = pd.read_csv('/content/dataset/mushroom_cleaned.csv')
dataset

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
0,1372,2,2,10,3.807467,1545,11,1.804273,1.0
1,1461,2,2,10,3.807467,1557,11,1.804273,1.0
2,1371,2,2,10,3.612496,1566,11,1.804273,1.0
3,1261,6,2,10,3.787572,1566,11,1.804273,1.0
4,1305,6,2,10,3.711971,1464,11,0.943195,1.0
...,...,...,...,...,...,...,...,...,...
17858,327,2,1,10,1.273702,587,11,1.804273,0.0
17859,307,2,1,10,1.174227,531,11,1.804273,0.0
17860,235,6,1,10,1.206059,516,11,0.943195,0.0
17861,248,6,1,10,1.245849,485,11,1.804273,0.0


Выполняет присваивание переменным X и y значений из загруженного набора данных.

Переменная X содержит матрицу признаков, которая представляет собой все столбцы данных, кроме последнего. Это обычно означает, что в X содержатся все признаки, которые мы будем использовать для обучения модели.

Переменная y содержит зависимую переменную, которая обычно является тем, что мы пытаемся предсказать с использованием модели машинного обучения. В данном случае, поскольку y соответствует последнему столбцу данных.

Каждая строка кода использует метод .iloc[], который позволяет выбирать данные по их целочисленным позициям в таблице данных.

Вывод на экран с помощью print() используется для отображения содержимого переменных X и y, что помогает визуально проверить правильность выборки данных перед тем, как перейти к следующему этапу анализа или обучению модели.

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

print("Матрица признаков"); print(X)
print("Зависимая переменная"); print(y)

Матрица признаков
[[1.37200000e+03 2.00000000e+00 2.00000000e+00 ... 1.54500000e+03
  1.10000000e+01 1.80427271e+00]
 [1.46100000e+03 2.00000000e+00 2.00000000e+00 ... 1.55700000e+03
  1.10000000e+01 1.80427271e+00]
 [1.37100000e+03 2.00000000e+00 2.00000000e+00 ... 1.56600000e+03
  1.10000000e+01 1.80427271e+00]
 ...
 [2.35000000e+02 6.00000000e+00 1.00000000e+00 ... 5.16000000e+02
  1.10000000e+01 9.43194554e-01]
 [2.48000000e+02 6.00000000e+00 1.00000000e+00 ... 4.85000000e+02
  1.10000000e+01 1.80427271e+00]
 [3.31000000e+02 2.00000000e+00 1.00000000e+00 ... 6.65000000e+02
  1.00000000e+00            nan]]
Зависимая переменная
[ 1.  1.  1. ...  0.  0. nan]


Выполняет проверку наличия пропущенных значений в данных, загруженных из файла CSV.

dataset.isnull().sum() используется для подсчета количества пропущенных значений в каждом столбце. Результат представлен в виде серии Pandas, где каждый столбец представлен вместе с количеством пропущенных значений в этом столбце.

В данном случае, результат показывает, что столбцы season и class содержат по одному пропущенному значению (представлены числами 1).

In [ ]:
# Обработка пропущенных значений
# Проверим, есть ли пропущенные значения в данных
print(dataset.isnull().sum())

cap-diameter       0
cap-shape          0
gill-attachment    0
gill-color         0
stem-height        0
stem-width         0
stem-color         0
season             1
class              1
dtype: int64


Переменная y идентифицируется как зависимая переменная, которую мы обычно пытаемся предсказать с использованием модели машинного обучения.

Сначала выводится исходное состояние переменной y.

Затем создается экземпляр LabelEncoder под названием labelencoder_y.

fit_transform() применяется к переменной y, чтобы преобразовать ее к числовому формату. fit_transform() выполняет два действия: сначала учит LabelEncoder категории, а затем преобразует их в числовое представление.

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder_y = LabelEncoder()
print('Зависимая переменная до обработки')
print(y)
y = labelencoder_y.fit_transform(y)
print('Зависимая переменная после обрабокти')
print(y)

Зависимая переменная до обработки
[ 1.  1.  1. ...  0.  0. nan]
Зависимая переменная после обрабокти
[1 1 1 ... 0 0 2]


Сначала, используя LabelEncoder, категориальный признак X[:, 0] (первый столбец данных) кодируется в числовой формат. Это делается для того, чтобы преобразовать категории в числа, что необходимо для использования OneHotEncoder.

Затем создается объект ColumnTransformer, который определяет, какие столбцы должны быть преобразованы с помощью OneHotEncoder, а какие должны остаться без изменений. В данном случае, OneHotEncoder применяется к первому столбцу (индекс 0), а остальные столбцы остаются без изменений.

OneHotEncoder выполняет преобразование категориальных признаков в бинарные векторы (One-Hot кодирование). Это означает, что каждая категория превращается в новый бинарный признак, который указывает на принадлежность объекта к этой категории.

В конечном итоге, первый столбец (который был закодирован LabelEncoder) заменяется One-Hot закодированными признаками, а затем удаляется, чтобы избежать проблемы мультиколлинеарности (линейной зависимости между признаками).

Выводится результат One-Hot кодирования.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])

ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [0])],
    remainder='passthrough'
)
X = ct.fit_transform(X)
X = X[:, 1:]
print("One-hot encoded функция:")
print(X)

One-hot encoded функция:
  (0, 1352)	1.0
  (0, 1598)	2.0
  (0, 1599)	2.0
  (0, 1600)	10.0
  (0, 1601)	3.8074667544799383
  (0, 1602)	1545.0
  (0, 1603)	11.0
  (0, 1604)	1.804272708628173
  (1, 1427)	1.0
  (1, 1598)	2.0
  (1, 1599)	2.0
  (1, 1600)	10.0
  (1, 1601)	3.8074667544799383
  (1, 1602)	1557.0
  (1, 1603)	11.0
  (1, 1604)	1.804272708628173
  (2, 1351)	1.0
  (2, 1598)	2.0
  (2, 1599)	2.0
  (2, 1600)	10.0
  (2, 1601)	3.6124962945838073
  (2, 1602)	1566.0
  (2, 1603)	11.0
  (2, 1604)	1.804272708628173
  (3, 1254)	1.0
  :	:
  (17859, 1604)	1.804272708628173
  (17860, 234)	1.0
  (17860, 1598)	6.0
  (17860, 1599)	1.0
  (17860, 1600)	10.0
  (17860, 1601)	1.2060593571348743
  (17860, 1602)	516.0
  (17860, 1603)	11.0
  (17860, 1604)	0.9431945538974952
  (17861, 247)	1.0
  (17861, 1598)	6.0
  (17861, 1599)	1.0
  (17861, 1600)	10.0
  (17861, 1601)	1.2458492469095952
  (17861, 1602)	485.0
  (17861, 1603)	11.0
  (17861, 1604)	1.804272708628173
  (17862, 330)	1.0
  (17862, 1598)	2.0
  (17862,

Код выполняет разделение данных на обучающий и тестовый наборы для обучения модели машинного обучения и её последующей оценки.

X и y: X представляет собой матрицу признаков, а y - вектор целевых переменных. Эти данные используются для обучения модели, где X содержит признаки, а y - соответствующие значения, которые мы пытаемся предсказать с использованием модели.

test_size=0.2: Этот аргумент указывает размер тестового набора данных. Здесь он равен 0.2, что означает, что 20% данных будут отложены для тестирования, а остальные 80% будут использоваться для обучения модели.

random_state=0: Этот параметр обеспечивает воспроизводимость разделения данных. Если указано конкретное значение (например, 0), то при каждом запуске кода разделение будет происходить одинаковым образом. Это полезно для того, чтобы результаты были воспроизводимыми и не менялись при повторных запусках кода.

X_train, X_test, y_train, y_test: После выполнения кода, данные будут разделены на четыре части:

X_train: Обучающий набор признаков, который будет использоваться для обучения модели.

X_test: Тестовый набор признаков, который будет использоваться для оценки производительности обученной модели.

y_train: Значения целевой переменной, соответствующие обучающему набору данных.

y_test: Значения целевой переменной, соответствующие тестовому набору данных.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

После разделения данных на обучающий и тестовый наборы с помощью train_test_split, следующим шагом обычно будет обучение модели на обучающих данных.

Инициализация модели: Сначала создается экземпляр линейной регрессии с помощью LinearRegression(). Это инициализирует модель линейной регрессии, которая будет обучена на данных.

Обучение модели: Затем вызывается метод fit(X_train, y_train), который обучает модель на обучающих данных. X_train представляет собой матрицу обучающих признаков, а y_train - вектор обучающих целевых переменных. В результате этой операции модель "изучает" зависимость между признаками и целевой переменной.


In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

Преобразование разреженной матрицы в плотную:

X_train.toarray(): Этот метод преобразует разреженную матрицу X_train (вероятно, созданную в результате применения метода OneHotEncoder или подобного метода к категориальным признакам) в плотную матрицу. Плотная матрица содержит все элементы и не содержит нулевых значений, в отличие от разреженной матрицы, где большинство значений равно нулю.

Добавление константы:

sm.add_constant(X_train_dense): Эта функция добавляет столбец констант (обычно единицы) к матрице X_train_dense. Это необходимо для того, чтобы модель линейной регрессии могла учитывать смещение (интерсепт) при обучении.

Печать преобразованных данных:

print(X_train_dense): Здесь просто выводится преобразованная матрица X_train_dense, вероятно, для проверки правильности преобразования.

In [ ]:
X_train_dense = X_train.toarray()
X_train_dense = sm.add_constant(X_train_dense)
print(X_train_dense)

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.04500000e+03
  1.10000000e+01 8.88450288e-01]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.30000000e+01
  7.00000000e+00 9.43194554e-01]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.48000000e+02
  1.20000000e+01 9.43194554e-01]
 ...
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.28000000e+02
  6.00000000e+00 8.88450288e-01]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 4.70000000e+01
  6.00000000e+00 8.88450288e-01]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.64800000e+03
  1.10000000e+01 9.43194554e-01]]


Выполняет множество шагов для анализа данных и построения модели линейной регрессии в рамках метода статистического анализа.

Модель линейной регрессии:

sm.OLS: Создает объект модели линейной регрессии методом наименьших квадратов (Ordinary Least Squares, OLS). Этот метод используется для оценки параметров линейной регрессии путем минимизации суммы квадратов остатков между наблюдаемыми и предсказанными значениями зависимой переменной.

endog=y_train: Задает зависимую переменную (целевую переменную) модели, которая обычно представляет собой вектор значений, которые мы пытаемся предсказать.

exog=X_train_dense: Задает матрицу независимых переменных (признаков), которые используются для предсказания зависимой переменной. Здесь X_train_dense - это матрица признаков, подготовленная на предыдущем этапе.

Обучение модели:

fit(): Метод fit() обучает модель линейной регрессии на предоставленных данных. В результате обучения модель находит оптимальные коэффициенты для каждого признака, которые минимизируют сумму квадратов остатков.

Вывод результатов:

regressor_OLS.summary(): Выводит сводку результатов анализа, включая различные статистические метрики, коэффициенты регрессии, p-значения и доверительные интервалы для коэффициентов. Это позволяет оценить значимость и влияние каждого признака на зависимую переменную.

In [ ]:
regressor_OLS = sm.OLS(endog=y_train, exog=X_train_dense).fit()
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     3.868
Date:                Tue, 23 Apr 2024   Prob (F-statistic):               0.00
Time:                        09:03:16   Log-Likelihood:                -7504.0
No. Observations:               14290   AIC:                         1.815e+04
Df Residuals:                   12719   BIC:                         3.004e+04
Df Model:                        1570                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6965      0.434      1.603      0.1

## **Вывод:**

Подготовка данных:

Изначально данные загружаются из файла CSV с помощью pd.read_csv(), после чего разделяются на матрицу признаков X и вектор целевой переменной y.

Проводится обработка пропущенных значений с помощью isnull().sum() для выявления пропусков в данных.

Применяется кодирование меток с помощью LabelEncoder для преобразования категориальных переменных в числовые значения.

Построение модели линейной регрессии:

Данные разбиваются на обучающий и тестовый наборы с использованием train_test_split().

Создается экземпляр класса LinearRegression() для модели линейной регрессии, который обучается на обучающих данных с помощью метода fit().

Использование статистического анализа с помощью statsmodels:

Проводится преобразование категориальных переменных в бинарные признаки с использованием OneHotEncoder и ColumnTransformer.

Для анализа модели используется statsmodels.api.OLS, который предоставляет дополнительную статистическую информацию о коэффициентах регрессии, p-значениях и доверительных интервалах.



### 1. Что такое кластерный анализ?

**Кластерный анализ** — это метод статистического анализа, который группирует набор объектов в кластеры так, чтобы объекты внутри одного кластера были более похожи друг на друга, чем на объекты из других кластеров. Этот метод используется для выявления структур в данных, упрощения анализа и визуализации больших объемов информации.

### 2. Перечислите известные методы кластерного анализа.

Некоторые известные методы кластерного анализа включают:

- **K-средние (K-means)**: Алгоритм, который делит данные на \( k \) кластеров, минимизируя внутрикластерные расстояния.
- **Иерархическая кластеризация**: Создает иерархическую структуру кластеров, используя агломеративные или дивизионные подходы.
- **DBSCAN (Density-Based Spatial Clustering of Applications with Noise)**: Основывается на плотности точек, позволяет находить кластеры произвольной формы и выявлять выбросы.
- **Mean Shift**: Алгоритм, который ищет области высокой плотности и перемещает центры кластеров в их направления.
- **Gaussian Mixture Models (GMM)**: Моделирует данные как смесь нескольких гауссовских распределений.

### 3. Перечислите классы и функции Python, которые задействованы при реализации кластерного анализа.

В Python для реализации кластерного анализа используются следующие классы и функции, в основном из библиотеки `scikit-learn`:

- **KMeans**: Для алгоритма K-средние.
- **AgglomerativeClustering**: Для иерархической кластеризации.
- **DBSCAN**: Для алгоритма DBSCAN.
- **MeanShift**: Для алгоритма Mean Shift.
- **GaussianMixture**: Для работы с гауссовыми смешанными моделями.
- **SilhouetteScore**: Для оценки качества кластеризации.

### 4. Опишите принцип определения оптимального количества кластеров.

Оптимальное количество кластеров можно определить с помощью различных методов:

- **Метод локтя**: Строится график зависимости суммы квадратов расстояний (inertia) от количества кластеров. Оптимальное количество кластеров соответствует "изгибу" на графике, где добавление новых кластеров не приводит к значительному уменьшению ошибки.
  
- **Силуэтный метод**: Вычисляет силуэт для каждого объекта, который показывает, насколько хорошо объект соответствует своему кластеру по сравнению с другими кластерами. Оптимальное количество кластеров соответствует максимальному среднему значению силуэтов.

- **Метод Гапа**: Сравнивает внутрикластерное расстояние с расстоянием, полученным от случайно распределенных точек.

### 5. Опишите принципиальные отличия методов регрессии, кластеризации и классификации.

- **Регрессия**:
  - Цель: Предсказать числовое значение (непрерывная переменная).
  - Примеры: Линейная регрессия, полиномиальная регрессия.
  - Применение: Моделирование зависимостей, прогнозирование.

- **Кластеризация**:
  - Цель: Группировка объектов в кластеры на основе их схожести.
  - Примеры: K-средние, иерархическая кластеризация.
  - Применение: Анализ данных, сегментация клиентов.

- **Классификация**:
  - Цель: Определить категорию (класс) для новых объектов на основе обученной модели.
  - Примеры: Логистическая регрессия, деревья решений, SVM.
  - Применение: Распознавание образов, фильтрация спама, медицинская диагностика.

Каждый из этих методов служит своей цели и используется в зависимости от типа задачи и данных.